<a href="https://colab.research.google.com/github/alphadev3296/deep-learning-practice/blob/main/Chapter-07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Char RNN - 1

import torch
from torch import nn, optim
import numpy as np

input_str = "apple"
label_str = "pple!"
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
print(f"vocab_size: {vocab_size}")

input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(f"training data size: {X.shape}")
print(f"label data size: {Y.shape}")

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(input_size, hidden_size, output_size)
outputs = net(X)
print(outputs.shape)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, input_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  result = outputs.data.numpy().argmax(axis=2)
  result_str = "".join([index_to_char[c] for c in np.squeeze(result)])
  print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

vocab_size: 5
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]
[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
training data size: torch.Size([1, 5, 5])
label data size: torch.Size([1, 5])
torch.Size([1, 5, 5])
0 loss:  1.7132222652435303 prediction:  [[2 2 1 1 1]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  eeaaa
1 loss:  1.4348585605621338 prediction:  [[4 4 4 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppp!
2 loss:  1.2238703966140747 prediction:  [[4 4 4 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppp!
3 loss:  1.0158822536468506 prediction:  [[4 4 4 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppl!
4 loss:  0.8301118612289429 prediction:  [[4 4 3 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppl!!
5 loss:  0.6692114472389221 prediction:  [[4 4 3 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction st

In [28]:
# Char RNN - 2

import torch
from torch import nn, optim

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

dic_size = len(char_dic)
print(f"char dic size: {dic_size}")

# set hyper parameters
hidden_size = dic_size
sequence_len = 10 # random number
learning_rate = 0.1

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_len):
  x_str = sentence[i:i+sequence_len]
  y_str = sentence[i+1:i+sequence_len+1]
  print(i, x_str, "->", y_str)

  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

print(x_data[0])
print(y_data[0])

x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(f"training data size: {X.shape}")
print(f"label size: {Y.shape}")

print(X[0])
print(Y[0])

class Net(nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(dic_size, hidden_size, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(f"output size: {outputs.shape}")

for i in range(300):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  if i % 10 == 0:
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
      if j == 0:
        predict_str += "".join([char_set[t] for t in result])
      else:
        predict_str += char_set[result[-1]]
    print(predict_str)

{'i': 0, ',': 1, 'k': 2, 'u': 3, 'e': 4, '.': 5, ' ': 6, 'c': 7, 's': 8, 'b': 9, 'm': 10, 'p': 11, 'g': 12, 'f': 13, 'r': 14, 'o': 15, 'h': 16, 'y': 17, 'l': 18, 't': 19, 'w': 20, 'a': 21, 'd': 22, 'n': 23, "'": 24}
char dic size: 25
0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  do